## Schema Registry

The schema registry allows the creation and retrieval of schemas. Deleting schemas is not possible. Updates can be done by by creating a modified copy of the schema with an updated version

In [ ]:
import os
import time
import datetime
import base64
import uuid
from pprint import pprint
from dotenv import load_dotenv
from typing import Any, Dict, Optional, Union, cast

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.models import VerifiableCredentialSchema
from prism_agent_open_api_specification_client.models import VerificationCredentialSchemaInput
from prism_agent_open_api_specification_client.models import InternalServerError
from prism_agent_open_api_specification_client.types import Response
from prism_agent_open_api_specification_client.api.schema_registry import create_schema, get_schema_by_id, lookup_schemas_by_query


### Utilitary Functions

In [ ]:
def print_schema(schema):
    if hasattr(schema, "id"):
        print("id:", schema.id)
    print("name:", schema.name)
    print("version:", schema.version)
    print("author:", schema.author)
    print("authored:", schema.authored)
    print("tags:", schema.tags)
    print("description:", schema.description)
    print("attributes:", schema.attributes)
    
def print_schema_page(schema_page):
    for schema in schema_page.contents:
        print_schema(schema)
        print()

### Client instances

For this example we only need one Client.

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [ ]:
load_dotenv("../BetaProgram/variables.env")
issuerApiKey = os.getenv('ISSUER_APIKEY')
issuerUrl = os.getenv('ISSUER_URL')

issuer_client = Client(base_url=issuerUrl, headers={"apiKey": issuerApiKey})

print(f"Issuer URL:{issuerUrl}")

### Create Schema

Publish the new schema with attributes to the schema registry on behalf of Cloud Agent. Schema will be signed by the keys of Cloud Agent and issued by the DID that corresponds to it

In [ ]:

data = {
    "name": f"Demo {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    "version": "1.0",
    "author": "John Doe",
    "description": "Demo schema",
    "attributes": [
        "type",
        "name"
    ],
    "authored": datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ"),
    "tags": [
        datetime.datetime.utcnow().strftime("%Y-%m-%d")
    ]
}

credential_schema_input = VerificationCredentialSchemaInput.from_dict(data)
credential_schema: Response[Union[InternalServerError, VerifiableCredentialSchema]] = create_schema.sync(client=issuer_client, json_body=credential_schema_input)
print(credential_schema)
print_schema(credential_schema)

### Lookup

Lookup schemas by `author`, `name`, `tags` parameters and control the pagination by `offset` and `limit` parameters

In [ ]:
schema_page = lookup_schemas_by_query.sync(client=issuer_client, tags=[datetime.datetime.utcnow().strftime("%Y-%m-%d")], limit=1)
print_schema_page(schema_page)

### Fetch
Fetch the schema by the unique identifier. Verifiable Credential Schema in json format is returned.

In [ ]:
credential_schema = get_schema_by_id.sync(client=issuer_client, id=credential_schema.id)
print_schema(credential_schema)